In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 700, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 700 (delta 101), reused 0 (delta 0), pack-reused 570
Receiving objects: 100% (700/700), 13.61 MiB | 19.57 MiB/s, done.
Resolving deltas: 100% (441/441), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 21.6 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps
from concurrent.futures import ThreadPoolExecutor

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/merging-models/tuning_hybrid_merging.db
/kaggle/input/merging-models/best_params_hybrid_merging.json
/kaggle/input/merging-models/__results__.html
/kaggle/input/merging-models/submission_merging_knn_rp3.csv
/kaggle/input/merging-models/__notebook__.ipynb
/kaggle/input/merging-models/__output__.json
/kaggle/input/merging-models/custom.css
/kaggle/input/merging-models/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/merging-models/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/merging-models/Recsys_Challenge_2023/LICENSE
/kaggle/input/merging-models/Recsys_Challenge_2023/.gitignore
/kaggle/input/merging-models/Recsys_Challenge_2023/rp3-tuning.ipynb
/kaggle/input/merging-models/Recsys_Challenge_2023/README.md
/kaggle/input/merging-models/Recsys_Challenge_2023/run_hyperparameter_search.py
/kaggle/input/merging-models/Recsys_Challenge_2023/cython_example_SLIM_MSE.py
/kaggle/input/merging-models/Recsys_Challenge_2023/requirements.txt
/kaggle/input/mergin

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid_merging.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.513297287731911, 'beta': 0.14806602610935488, 'topK': 54}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
    params_user_knn = json.load(params_file)
params_user_knn = {"similarity": "cosine", "topK": 506, "shrink": 0}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [17]:
recommender_item = ItemKNNCFRecommender(URM_train)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_train)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_train)
recommender_rp3.fit(**params_rp3)

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_train, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4599.92 column/sec. Elapsed time 4.86 sec
UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4570.22 column/sec. Elapsed time 2.85 sec
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2878.44 column/sec. Elapsed time 7.76 sec
ItemKNNSimilarityHybridRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 247 ( 1.1%) items with no interactions.


In [18]:
class RoundRobinListCombinationRecommender:
    def __init__(self, URM_train, recommenders):
        # recommenders must be pretrained
        self.URM_train = URM_train
        self.recommenders = recommenders
        
    def recommend(self, user_ids, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=False, return_scores=True, remove_custom_items_flag=True):
        
        final_recommendations = []
        final_scores = []
        for user_id in user_ids:
            
            all_recommendations = []
            all_scores = []
            for recommender in self.recommenders:
                recommendations, scores = recommender.recommend(
                    [user_id],
                    cutoff=cutoff,
                    remove_seen_flag=remove_seen_flag, 
                    remove_top_pop_flag=True,
                    return_scores=True, 
                    remove_custom_items_flag=remove_custom_items_flag
                )
                all_recommendations.append(recommendations[0])
                all_scores.append(scores[0])
               
            recommendations = []
            current_local_index = 0
            while len(recommendations) < cutoff:
                for r in all_recommendations:
                    if len(recommendations) < cutoff and r[current_local_index] not in recommendations:
                        recommendations.append(r[current_local_index])
                current_local_index += 1
                
            final_recommendations.append(recommendations)
            final_scores.append(all_scores[0])
        if return_scores:
            return np.array(final_recommendations), np.array(final_scores)
        return np.array(final_recommendations)
    
    def get_URM_train(self):
        return self.URM_train

In [19]:
list_combinator = RoundRobinListCombinationRecommender(URM_train, [recommender_item, recommender_rp3])

In [20]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

result_df, _ = evaluator.evaluateRecommender(list_combinator)
display(result_df)
result_df, _ = evaluator.evaluateRecommender(hybrid_similarity)
display(result_df)
result_df, _ = evaluator.evaluateRecommender(recommender_item)
display(result_df)
result_df, _ = evaluator.evaluateRecommender(recommender_rp3)
display(result_df)
result_df, _ = evaluator.evaluateRecommender(recommender_user)
display(result_df)

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 16.37 sec. Users per second: 639


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.095849,0.166454,0.140554,0.049167,0.084245,0.271527,0.151478,0.113975,0.520945,0.360546,...,0.802826,0.418228,0.802826,0.072305,10.352576,0.99766,0.20863,0.797522,1.797217,0.303019


EvaluatorHoldout: Processed 10456 (100.0%) in 7.75 sec. Users per second: 1350


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096882,0.167023,0.140856,0.049977,0.085718,0.274123,0.153088,0.114802,0.521136,0.365169,...,0.802826,0.418381,0.802826,0.074617,10.464487,0.998049,0.2153,0.806143,1.677631,0.304115


EvaluatorHoldout: Processed 10456 (100.0%) in 7.36 sec. Users per second: 1420


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.09451,0.161146,0.135378,0.048719,0.082976,0.267977,0.148683,0.111312,0.511572,0.356662,...,0.802826,0.410703,0.802826,0.066309,10.283111,0.997789,0.191329,0.792171,1.734564,0.30426


EvaluatorHoldout: Processed 10456 (100.0%) in 7.77 sec. Users per second: 1345


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.093831,0.162868,0.137495,0.048469,0.083358,0.271221,0.149568,0.111542,0.515207,0.357898,...,0.802826,0.413621,0.802826,0.084954,10.632953,0.998201,0.245126,0.819121,1.595072,0.308564


EvaluatorHoldout: Processed 10456 (100.0%) in 11.52 sec. Users per second: 908


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.071136,0.128879,0.110422,0.034688,0.062624,0.221933,0.116597,0.086529,0.447781,0.27593,...,0.802826,0.35949,0.802826,0.010578,7.57523,0.984157,0.030522,0.583566,3.76594,0.257261


In [21]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

# recommender_user = UserKNNCFRecommender(URM_all)
# recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

# hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_all, recommender_item.W_sparse, recommender_rp3.W_sparse)
# hybrid_similarity.fit(**params_merging)

list_combinator = RoundRobinListCombinationRecommender(URM_train, [recommender_item, recommender_rp3])
recommendations = list_combinator.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())

submission_df.to_csv('/kaggle/working/submission_list_comb_knn_rp3.csv', index=False)

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 4289.32 column/sec. Elapsed time 5.21 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2636.36 column/sec. Elapsed time 8.48 sec


,user_id,item_list
0,1,101 36 403 506 515 123 694 592 977 314
1,2,1095 11 47 196 28 50 37 1522 4 2
2,3,59 239 857 259 511 1281 4252 584 375 750
3,4,28 249 50 139 7 145 146 129 171 612
4,5,1570 77 4 5138 116 1220 148 131 175 7033
